<h1>Federated Word Vectors</h1>

This example demonstrates how training a simple Image classifier written in PyTorch could be trained using federated learning with PySyft. We distribute the image data to two workers Bob and Alice to whom the model is sent and trained. Upon training the model the trained model is sent back to the owner of the model and used to make predictions. 


Hrishikesh Kamath - GitHub: @<a href="http://github.com/kamathhrishi">kamathhrishi</a>

In [93]:
#Import modules required for PyTorch Neural Networks

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset

In [94]:
#Define seed to maintain consistency 

torch.manual_seed(1)

In [95]:
#Import PySyft library required for federated learning

import syft as sy  

In [96]:
#Define Syft workers Bob and Alice for federated learning

hook = sy.TorchHook(torch)  # <-- NEW: hook PyTorch ie add extra functionalities to support Federated Learning
bob = sy.VirtualWorker(hook, id="bob")  # <-- NEW: define remote worker bob
alice = sy.VirtualWorker(hook, id="alice")  # <-- NEW: and alice

In [97]:


CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
EPOCHS=10
LR=0.001

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()


In [98]:
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [99]:
class TextDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,text,transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.text = text
        self.data=text
        self.onehot=[]
        self.targets=[]
        self.transform = transform
        self.create_trigrams()

    def __len__(self):
        
        return len(self.onehot)
    
    def create_trigrams(self):
        
        self.trigrams = [([self.text[i], self.text[i + 1]],self.text[i + 2])
            for i in range(len(self.text) - 2)]
        
        for words,target in trigrams:
            
            tensor=torch.tensor([word_to_ix[w] for w in words],dtype=torch.long)
            self.onehot.append(tensor)
            self.targets.append(torch.tensor([word_to_ix[target]], dtype=torch.long))

    def __getitem__(self, idx):
        
        sample=self.onehot[idx]
        target=self.targets[idx]
                
        if self.transform:
            sample = self.transform(sample)

        return sample,target

In [100]:
federated_train_loader = sy.FederatedDataLoader( # <-- this is now a FederatedDataLoader 
                         TextDataset(test_sentence)
                         .federate((bob, alice)),batch_size=1)

Scanning and sending data to bob, alice...
Done!


In [101]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size,embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        
        return log_probs

In [102]:
losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=LR)


In [103]:
def train():
    
    model.train()
    for context, target in federated_train_loader:
        
        model.send(context.location)
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = context
    
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs,target[0])

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()
        model.get()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        loss.get()
        
    losses.append(total_loss)
    
    

In [104]:

for epoch in range(0,EPOCHS):
    total_loss = 0
    print("EPOCH: ",epoch+1)
    train()
    
print(losses)  # The loss decreased every iteration over the training data!

EPOCH:  1


/Users/hrishikesh/anaconda3/envs/syft_1/lib/python3.6/site-packages/syft/frameworks/torch/tensors/interpreters/native.py:215: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  response = eval(cmd)(*args, **kwargs)


EPOCH:  2
EPOCH:  3
EPOCH:  4
EPOCH:  5
EPOCH:  6
EPOCH:  7
EPOCH:  8
EPOCH:  9
EPOCH:  10
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [105]:
ref_index=3
ref=model.embeddings(torch.LongTensor([ref_index]))

In [106]:
index=None
Similarity=0.0

for i in range(0,97):
   
   if(i!=ref_index):
        
      cos = nn.CosineSimilarity(dim=1, eps=1e-6)
      output = cos(ref,model.embeddings(torch.LongTensor([i])))
   
    
      if(output.item()>float(Similarity)):
        
          Similarity=output
          index=i